In [1]:
import re
from pathlib import Path

import numpy as np
from Bio import SeqIO, Seq
from matplotlib import pyplot as plt

In [2]:
myseqfile = "/home/alex/shared_folder/SCerevisiae/results/generated/4kb_lowpol_10seq_randomflanks_v8/concat40kb_lowpol_norna.fa"
manonseqfile = "/home/alex/shared_folder/SCerevisiae/data/SynC 1.fa"

In [3]:
myseq = list({seq.id: str(seq.seq) for seq in SeqIO.parse(myseqfile, format="fasta")}.values())[0]
manonseq = list({seq.id: str(seq.seq) for seq in SeqIO.parse(manonseqfile, format="fasta")}.values())[0]

In [4]:
mutpos = []
cpt = 0
for i in range(len(myseq)):
    if myseq[i] != manonseq[i]:
        print(i, myseq[i], manonseq[i])
        mutpos.append(i)
        cpt += 1
print(cpt)

518 G T
522 C G
1713 A G
1714 G A
1716 C G
5626 A G
5627 G A
5629 C G
7178 G T
7182 C G
8937 A G
8938 G A
8940 C G
13445 G T
13449 C G
14605 G T
14609 C G
17217 A G
17218 G A
17220 C G
18864 C G
18865 G T
18869 C G
20236 G T
20240 C G
30691 G T
30695 C G
32090 G T
32094 C G
33105 G T
33109 C G
33856 C G
33857 G T
33861 C G
34133 A G
34134 G A
34137 G C
34564 G T
34568 C G
34918 A G
34919 G A
34922 G C
36407 A G
36408 G A
36410 C G
38773 A G
38774 G A
38776 C G
48


In [5]:
from Modules import utils
utils.RCdna("GGTCTC")

'GAGACC'

In [6]:
def find_subseq(subseq, seq):
    subseq_rev = utils.RCdna(subseq)
    pattern = f"(?P<fwd>{subseq[0]}(?={subseq[1:]}))|(?P<rev>{subseq_rev[0]}(?={subseq_rev[1:]}))"
    sites = [(m.start(), "fwd" if m.groups()[0] is not None else "rev") for m in re.finditer(pattern, seq)]
    return sites

In [7]:
for subseq in ["GGTCTC", "CGTCTC", "GCGGCCGC"]:
    print(subseq)
    sites = find_subseq(subseq, myseq)
    print("myseq", sites)
    sites = find_subseq(subseq, manonseq)
    print("manonseq", sites)

GGTCTC
myseq [(517, 'fwd'), (1712, 'rev'), (5625, 'rev'), (7177, 'fwd'), (8936, 'rev'), (13444, 'fwd'), (14604, 'fwd'), (17216, 'rev'), (20235, 'fwd'), (30690, 'fwd'), (32089, 'fwd'), (33104, 'fwd'), (34563, 'fwd'), (36406, 'rev'), (38772, 'rev')]
manonseq []
CGTCTC
myseq [(18864, 'fwd'), (33856, 'fwd'), (34132, 'rev'), (34917, 'rev')]
manonseq []
GCGGCCGC
myseq []
manonseq []


In [8]:
bsa = find_subseq("GGTCTC", myseq)
bsmb = find_subseq("CGTCTC", myseq)
merge = np.array(bsa + bsmb)[:, 0].astype(int)
merge = np.sort(merge)
merge

array([  517,  1712,  5625,  7177,  8936, 13444, 14604, 17216, 18864,
       20235, 30690, 32089, 33104, 33856, 34132, 34563, 34917, 36406,
       38772])

In [9]:
mutpos = np.array(mutpos)
mutpos

array([  518,   522,  1713,  1714,  1716,  5626,  5627,  5629,  7178,
        7182,  8937,  8938,  8940, 13445, 13449, 14605, 14609, 17217,
       17218, 17220, 18864, 18865, 18869, 20236, 20240, 30691, 30695,
       32090, 32094, 33105, 33109, 33856, 33857, 33861, 34133, 34134,
       34137, 34564, 34568, 34918, 34919, 34922, 36407, 36408, 36410,
       38773, 38774, 38776])

In [11]:
site_to_pos = {site: [] for site in merge}
for pos in mutpos:
    site_idx = np.searchsorted(merge, pos, side='right') - 1
    site = merge[site_idx]
    site_to_pos[site].append(pos - site)
all_sites = {
    **{site: f"bsa-{sense}" for site, sense in bsa},
    **{site: f"bsmb-{sense}" for site, sense in bsmb}
}
for site, relpos in site_to_pos.items():
    print(site, relpos, all_sites[site], myseq[site:site+6])

517 [1, 5] bsa-fwd GGTCTC
1712 [1, 2, 4] bsa-rev GAGACC
5625 [1, 2, 4] bsa-rev GAGACC
7177 [1, 5] bsa-fwd GGTCTC
8936 [1, 2, 4] bsa-rev GAGACC
13444 [1, 5] bsa-fwd GGTCTC
14604 [1, 5] bsa-fwd GGTCTC
17216 [1, 2, 4] bsa-rev GAGACC
18864 [0, 1, 5] bsmb-fwd CGTCTC
20235 [1, 5] bsa-fwd GGTCTC
30690 [1, 5] bsa-fwd GGTCTC
32089 [1, 5] bsa-fwd GGTCTC
33104 [1, 5] bsa-fwd GGTCTC
33856 [0, 1, 5] bsmb-fwd CGTCTC
34132 [1, 2, 5] bsmb-rev GAGACG
34563 [1, 5] bsa-fwd GGTCTC
34917 [1, 2, 5] bsmb-rev GAGACG
36406 [1, 2, 4] bsa-rev GAGACC
38772 [1, 2, 4] bsa-rev GAGACC


In [191]:
sites = find_subseq("CGTCTC", myseq)
sites

[(18864, 'fwd'), (33856, 'fwd'), (34132, 'rev'), (34917, 'rev')]

In [192]:
for pos, sense in sites:
    print(myseq[pos:pos+6], manonseq[pos:pos+6])

CGTCTC GTTCTG
CGTCTC GTTCTG
GAGACG GGAACC
GAGACG GGAACC
